In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

def create_and_train_model():
    # Load MNIST dataset
    mnist = tf.keras.datasets.mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    
    # Normalize the data
    x_train = tf.keras.utils.normalize(x_train, axis=2)
    x_test = tf.keras.utils.normalize(x_test, axis=2)
    
    # Reshape the data
    x_trainr = np.array(x_train).reshape(-1, 28, 28, 1)
    x_testr = np.array(x_test).reshape(-1, 28, 28, 1)
    
    # Create model
    model = Sequential()
    
    # Add layers
    model.add(Conv2D(64, (3,3), input_shape=x_trainr.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(64, (3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(64, (3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation("relu"))

    model.add(Dense(32))
    model.add(Activation("relu"))

    model.add(Dense(10))
    model.add(Activation("softmax"))
    
    # Compile model
    model.compile(loss="sparse_categorical_crossentropy", 
                 optimizer="adam", 
                 metrics=['accuracy'])
    
    # Train model
    model.fit(x_trainr, y_train, epochs=10, validation_split=0.3)
    
    return model

class DigitRecognizerGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Digit  Image Recognition System")
        self.root.geometry("800x600")
        
        # Create and train model
        self.load_or_train_model()
        
        # Create GUI elements
        self.create_widgets()
        
    def load_or_train_model(self):
        try:
            self.model = tf.keras.models.load_model('digit_recognition_model.h5')
            messagebox.showinfo("Success", "Model loaded successfully!")
        except:
            messagebox.showinfo("Info", "Training new model... This may take a few minutes.")
            self.model = create_and_train_model()
            self.model.save('digit_recognition_model.h5')
            messagebox.showinfo("Success", "Model trained and saved successfully!")
            
    def create_widgets(self):
        # Create main frame
        self.main_frame = tk.Frame(self.root, padx=20, pady=20)
        self.main_frame.pack(expand=True, fill='both')
        
        # Title
        title_label = tk.Label(self.main_frame, 
                             text="Digit Image Recognition System",
                             font=("Arial", 24, "bold"))
        title_label.pack(pady=20)
        
        # Button to select image
        self.select_button = tk.Button(self.main_frame,
                                     text="Select Image",
                                     command=self.select_image,
                                     font=("Arial", 12),
                                     width=20)
        self.select_button.pack(pady=10)
        
        # Create frame for displaying images
        self.image_frame = tk.Frame(self.main_frame)
        self.image_frame.pack(pady=20)
        
        # Result label
        self.result_label = tk.Label(self.main_frame,
                                   text="Recognized Digit: ",
                                   font=("Arial", 16))
        self.result_label.pack(pady=10)
        
    def select_image(self):
        # Open file dialog
        file_path = filedialog.askopenfilename(
            filetypes=[("Image files", "*.png *.jpg *.jpeg *.bmp *.gif *.tiff")])
        
        if file_path:
            self.process_image(file_path)
            
    def process_image(self, image_path):
        # Read and process image
        img2 = cv2.imread(image_path)
        ret, img1 = cv2.threshold(img2, 75, 255, cv2.THRESH_BINARY)
        img = cv2.bitwise_not(img1)
        
        # Create figure for displaying images
        fig = Figure(figsize=(8, 3))
        
        # Original image
        ax1 = fig.add_subplot(131)
        ax1.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
        ax1.set_title("Original")
        ax1.axis('off')
        
        # Binary image
        ax2 = fig.add_subplot(132)
        ax2.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
        ax2.set_title("Binary")
        ax2.axis('off')
        
        # Inverted image
        ax3 = fig.add_subplot(133)
        ax3.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        ax3.set_title("Inverted")
        ax3.axis('off')
        
        # Remove old canvas if it exists
        for widget in self.image_frame.winfo_children():
            widget.destroy()
            
        # Create canvas and display figure
        canvas = FigureCanvasTkAgg(fig, master=self.image_frame)
        canvas.draw()
        canvas.get_tk_widget().pack()
        
        # Process image for prediction
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (28,28), interpolation=cv2.INTER_AREA)
        newimg = tf.keras.utils.normalize(resized, axis=1)
        newimg = np.array(newimg).reshape(-1, 28, 28, 1)
        
        # Make prediction
        predictions = self.model.predict(newimg)
        predicted_digit = np.argmax(predictions[0])
        
        # Update result label
        self.result_label.config(text=f"Recognized Digit: {predicted_digit}")

def main():
    root = tk.Tk()
    app = DigitRecognizerGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()
    model.save('digit_recognition_model.h5')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step


NameError: name 'model' is not defined